# Tiny Dataset notebook

In [3]:
# import sqlite3
# import pandas as pd
# import os
# import openai
# from freeplay import Freeplay 
# from dotenv import load_dotenv
# 
# load_dotenv()
# 
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
# FREEPLAY_API_KEY = os.environ.get("FREEPLAY_API_KEY")
# 
# fp_client = Freeplay(
#     provider_config=ProviderConfig(openai=OpenAIConfig(OPENAI_API_KEY)),
#     freeplay_api_key=FREEPLAY_API_KEY,
#     api_base=f'https://8thlight.freeplay.ai/api'
# )
# 
# prompt_vars = {"keyA": "valueA"}
# completion = fp_client.get_cpmpletion(project_id='AI_Cohort_1_Group_3_Explorations',
#                                       template_name="Evaluation of Summary Capabilities",
#                                       variables=prompt_vars)
# 
# 
# base_path = f"{os.getcwd()}"
# 
# file_path = os.path.join(os.path.dirname(base_path), 'data/ctg-studies-tiny.db')
# 
# conn = sqlite3.connect(file_path)
# 
# query = "SELECT * FROM tiny_dataset"
# df = pd.read_sql_query(query, conn)
# # data_summary = df.describe().to_string() use a summary of the data to pass to LM
# data = df.head()
# 
# 
# messages = 
# response = openai.chat.completions.create(
#     model="gpt-4-turbo",
#     messages=messages
# )
# 
# # Keeps the conversational context intact
# assistant_response = response.choices[0].message.content
# print(assistant_response)
# messages.append({
#     "role": "assistant",
#     "content": assistant_response
# })


The data provided is a summary of clinical studies with various attributes such as study ID, completion date, study type, eligibility, and demographic details among others. Here is a summarized overview of five specific clinical studies within the dataset:

1. **NCT00436514** (Sequence 286365)
   - **Completion**: July 2009 (Actual)
   - **Summary**: Focused on colorectal cancer, being a frequent cause of death particularly in Europe.
   - **Description**: Not provided.
   - **Condition**: Colorectal Cancer
   - **Study Type**: Interventional
   - **Eligibility**: Adults aged 18 to 80 years.
   - **Volunteers**: Not accepting healthy volunteers.
   - **Sex**: All
   - **Age Range**: 18 to 80 years
   - **References**: Includes a derived citation (PMID: 21039676).

2. **NCT02827617** (Sequence 150334)
   - **Completion**: Expected by December 31, 2024 (Estimated)
   - **Summary**: Aims to identify new prognostic factors for chronic lymphocytic leukemia.
   - **Description**: Focuses on 

### Continue the conversation
Replace the content of the `new_user_message` variable with different prompts to continue the conversation.

In [ ]:
# new_user_message = "Is there anything interesting I should know bout this data?"
# 
# messages.append({
#     "role": "user",
#     "content": new_user_message
# })
# 
# response = openai.chat.completions.create(
#     model="gpt-4-turbo",
#     messages=messages
# )
# assistant_response = response.choices[0].message.content
# print(assistant_response)
# messages.append({
#     "role": "assistant",
#     "content": assistant_response
# })

In [11]:
import time
import os
import openai
from freeplay import Freeplay, RecordPayload, CallInfo, ResponseInfo, SessionInfo, CallInfo
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")
OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
FREEPLAY_API_KEY = os.environ.get("FREEPLAY_API_KEY")


# create your a freeplay client object
fpClient = Freeplay(
    freeplay_api_key=os.getenv("FREEPLAY_API_KEY"),
    api_base="https://8thlight.freeplay.ai/api"
)

## PROMPT FETCH ##
# set the prompt variables
prompt_vars = {"keyA": "valueA"}
# get a formatted prompt
formatted_prompt = fpClient.prompts.get_formatted(project_id="473ef06b-2883-41da-85dd-9ccec95c95c6",
                                                  template_name="Evaluation of Summary Capabilities",
                                                  environment="latest",
                                                  variables=prompt_vars)

## LLM CALL ##
# Make an LLM call to your provider of choice
start = time.time()
chat_response =  openai.chat.completions.create(
    model=formatted_prompt.prompt_info.model,
    messages=formatted_prompt.messages,
    **formatted_prompt.prompt_info.model_parameters
)
end = time.time()

# add the response to your message set
all_messages = formatted_prompt.all_messages(
    {'role': chat_response.choices[0].message.role,
     'content': chat_response.choices[0].message.content}
)

## RECORD ##
# create a session
session = fpClient.sessions.create()

# build the record payload
payload = RecordPayload(
    all_messages=all_messages,
    inputs=prompt_vars,
    session_info=session,
    prompt_info=formatted_prompt.prompt_info,
    call_info=CallInfo.from_prompt_info(formatted_prompt.prompt_info, start_time=start, end_time=end),
    response_info=ResponseInfo(
        is_complete=chat_response.choices[0].finish_reason == 'stop'
    )
)
# record the LLM interaction
fpClient.recordings.create(payload)

#


RecordResponse(completion_id='7c83c38e-3c89-4490-bf92-491233ef6a03')